In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [2]:
df = sns.load_dataset("mpg")

X_train, y_train = df[~df["horsepower"].isna()][["displacement", "acceleration"]], df[~df["horsepower"].isna()]["horsepower"]
X_pred = df[df["horsepower"].isna()][["displacement", "acceleration"]]

linreg = LinearRegression()
linreg.fit(X_train, y_train)
y_pred = linreg.predict(X_pred)
y_pred = np.round(y_pred)
df.loc[X_pred.index, "horsepower"] = y_pred
df = pd.get_dummies(df.drop("name", axis = 1), columns = ["origin"])
df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin_europe,origin_japan,origin_usa
0,18.0,8,307.0,130.0,3504,12.0,70,0,0,1
1,15.0,8,350.0,165.0,3693,11.5,70,0,0,1
2,18.0,8,318.0,150.0,3436,11.0,70,0,0,1
3,16.0,8,304.0,150.0,3433,12.0,70,0,0,1
4,17.0,8,302.0,140.0,3449,10.5,70,0,0,1


In [3]:
X, y = df.drop(["mpg"], axis = 1).values, df["mpg"].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size = 0.5, random_state = 42)

In [4]:
scaler = StandardScaler()
#scaler = MinMaxScaler()

scaled_X_train = scaler.fit_transform(X_train)
scaled_X_val = scaler.transform(X_val)
scaled_X_test = scaler.transform(X_test)

In [12]:
sigmoid = np.vectorize(lambda x: 1 / (1 + math.exp(-x)))

def seed_np(integer):
    np.random.seed(integer)

class ERegressor:
    class Net:
        def __init__(self):
            pass

        def set_input_size(self, input_size):
            self.input_size = input_size
        
        def set_weights(self, weights_and_biases):
            self.w1 = weights_and_biases[0]
            self.w2 = weights_and_biases[1]
            self.w3 = weights_and_biases[2]
            self.b1 = weights_and_biases[3]
            self.b2 = weights_and_biases[4]
            self.b3 = weights_and_biases[5]
        
        def set_mutation_sigma(self, n):
            #self.mutation_sigma = 3 - (2 - 40 / (10 + n))
            self.mutation_sigma = 2.5 - (3 - 1200 / (200 + n))
        
        def predict(self, input):

            return (self.w3.T @ sigmoid(self.w2.T @ sigmoid(self.w1.T @ input.T + self.b1) + self.b2) + self.b3).reshape(-1)

        
        def __add__(self, other):
            w1 = (self.w1 + other.w1) / 2 + np.random.normal(0, self.mutation_sigma, (self.input_size, self.input_size))
            w2 = (self.w2 + other.w2) / 2 + np.random.normal(0, self.mutation_sigma, (self.input_size, self.input_size))
            w3 = (self.w3 + other.w3) / 2 + np.random.normal(0, self.mutation_sigma, (self.input_size, 1))
            b1 = (self.b1 + other.b1) / 2 + np.random.normal(0, self.mutation_sigma, (self.input_size, 1))
            b2 = (self.b2 + other.b2) / 2 + np.random.normal(0, self.mutation_sigma, (self.input_size, 1))
            b3 = (self.b3 + other.b3) / 2 + np.random.normal(0, self.mutation_sigma, 1)

            return w1, w2, w3, b1, b2, b3
    
    def __init__(self, n = 100, random_state = None):
        self.nets = {}
        self.y_preds = {}
        self.best_net = None
        self.best_result = None
        self.n = n // 2 * 2
        
        for i in range(self.n):
            self.nets[i] = self.Net()

        if random_state != None:
            seed_np(random_state)
    
    def fit(self, X_train, y_train, epochs = 100):
        self.input_size = X_train.shape[1]

        for key in self.nets.keys():
            self.nets[key].set_input_size(self.input_size)

            self.nets[key].set_weights((
                np.random.uniform(-3, 3, (self.input_size, self.input_size)),
                np.random.uniform(-3, 3, (self.input_size, self.input_size)),
                np.random.uniform(-3, 3, (self.input_size, 1)),
                np.random.uniform(-3, 3, (self.input_size, 1)),
                np.random.uniform(-3, 3, (self.input_size, 1)),
                np.random.uniform(-3, 3, 1))
            )

        for epoch in range(epochs):
            for key, _ in self.nets.items():
                self.y_preds[key] = self.nets[key].predict(X_train)
                self.nets[key].set_mutation_sigma(epoch)
            
            self.mean_absolute_errors = {}
            
            for key, _ in self.y_preds.items():
                self.mean_absolute_errors[key] = mean_absolute_error(y_train, self.y_preds[key])

            self.sorted_indecies = [key for key, _ in sorted(self.mean_absolute_errors.items(), key = lambda x: x[1])]

            if self.best_result != None:
                if self.best_result[1] > sorted(self.mean_absolute_errors.items(), key = lambda x: x[1])[0][1]:
                    self.best_result = sorted(self.mean_absolute_errors.items(), key = lambda x: x[1])[0]
                    print(f"Epoch {epoch}: MAE: {self.best_result[1]}")
            else:        
                self.best_result = sorted(self.mean_absolute_errors.items(), key = lambda x: x[1])[0]
                print(f"Epoch {epoch}: MAE: {self.best_result[1]}")

            self.best_net = self.sorted_indecies[0]
            
            for i in range(0, self.n // 2, 2):
                self.nets[self.sorted_indecies[self.n // 2 + i]].set_weights(self.nets[self.sorted_indecies[i]] + self.nets[self.sorted_indecies[i + 1]])
                self.nets[self.sorted_indecies[self.n // 2 + i + 1]].set_weights(self.nets[self.sorted_indecies[i]] + self.nets[self.sorted_indecies[i + 1]])
            

    def predict(self, X):
        return self.nets[self.best_net].predict(X)

In [13]:
regressor = ERegressor(n = 60, random_state = 42)
regressor.fit(scaled_X_train, y_train, epochs = 1000)

Epoch 0: MAE: 15.277470548058556
Epoch 1: MAE: 8.732734239990856
Epoch 2: MAE: 6.379045017328299
Epoch 3: MAE: 6.115675026516401
Epoch 4: MAE: 6.102236259960406
Epoch 5: MAE: 4.881021594951148
Epoch 9: MAE: 4.5606258520362015
Epoch 45: MAE: 4.099650116775665
Epoch 63: MAE: 3.8587145141165853
Epoch 87: MAE: 3.7782456902595736
Epoch 101: MAE: 3.6932572397557792
Epoch 132: MAE: 3.574721149799812
Epoch 141: MAE: 3.504528593136012
Epoch 144: MAE: 3.446711422859714
Epoch 175: MAE: 3.3800300972289468
Epoch 183: MAE: 3.289604937241284
Epoch 206: MAE: 3.2651029974231136
Epoch 217: MAE: 3.250262712047534
Epoch 227: MAE: 3.1085723727897854
Epoch 230: MAE: 3.075742120269175
Epoch 247: MAE: 3.072882735461881
Epoch 256: MAE: 2.812837259909907
Epoch 350: MAE: 2.746488815767935
Epoch 364: MAE: 2.73984070619458
Epoch 373: MAE: 2.701550354563329
Epoch 386: MAE: 2.6800531133005276
Epoch 400: MAE: 2.6607113705068084
Epoch 413: MAE: 2.6197881329730963
Epoch 436: MAE: 2.521761819233827
Epoch 487: MAE: 2.492

In [14]:
y_pred = regressor.predict(scaled_X_test)

In [15]:
mean_absolute_error(y_test, y_pred)

2.102977438053208